In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

is_dark = True
theme, cs = rp.mpl_setup(is_dark)

In [2]:
import pyvista as pv
pv.set_jupyter_backend('trame')

if is_dark:
    pv.set_plot_theme('dark')
else:
    pv.set_plot_theme('default')

In [155]:
import pyxtal as px

sg = px.Group('Fd-3c')
sg_ops = sg.Wyckoff_positions[0].ops
sg_ops

[SymmOp(affine_matrix=array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])),
 SymmOp(affine_matrix=array([[-1.  ,  0.  ,  0.  ,  0.25],
        [ 0.  , -1.  ,  0.  ,  0.75],
        [ 0.  ,  0.  ,  1.  ,  0.5 ],
        [ 0.  ,  0.  ,  0.  ,  1.  ]])),
 SymmOp(affine_matrix=array([[-1.  ,  0.  ,  0.  ,  0.75],
        [ 0.  ,  1.  ,  0.  ,  0.5 ],
        [ 0.  ,  0.  , -1.  ,  0.25],
        [ 0.  ,  0.  ,  0.  ,  1.  ]])),
 SymmOp(affine_matrix=array([[ 1.  ,  0.  ,  0.  ,  0.5 ],
        [ 0.  , -1.  ,  0.  ,  0.25],
        [ 0.  ,  0.  , -1.  ,  0.75],
        [ 0.  ,  0.  ,  0.  ,  1.  ]])),
 SymmOp(affine_matrix=array([[0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]])),
 SymmOp(affine_matrix=array([[ 0.  ,  0.  ,  1.  ,  0.5 ],
        [-1.  ,  0.  ,  0.  ,  0.25],
        [ 0.  , -1.  ,  0.  ,  0.75],
        [ 0.  ,  0.  ,  0.  ,  1.  ]])),
 SymmOp(affine_matrix=array([[ 0.  ,  0.

In [156]:

# lines = []
# for pl1 in planes:
#     for pl2 in planes:
#         intersection = pl1.intersection(pl2)
#         if not isinstance(intersection, g3.Line):
#             continue

#         lines.append(intersection)

        # p1, v = intersection.parametric()
        # p2 = p1 + v * 1

        # p.add_mesh(pv.Line(p1._v, p2._v), color=cs[1], opacity=0.5)

# int_points = []
# for l1 in lines:
#     for l2 in lines:
#         int_point = l1.intersection(l2)
#         if not isinstance(int_point, g3.Point):
#             continue

#         int_points.append(int_point)
#print(len(int_points))

In [157]:
import Geometry3D as g3
from scipy.spatial import HalfspaceIntersection, ConvexHull

pt = np.array([0.1, 0.3, 0.2])

p = pv.Plotter()

p.add_mesh(pv.Sphere(radius=0.05, center=pt), color=cs[0])

planes = []

shifts = np.array([
    [0, 0, 0],
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, -1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])

for op in sg_ops[1:]:
    for shift in shifts:
        image = op.operate(pt) % 1
        image += shift
        if np.allclose(image, pt) or np.sum(np.square(image - pt)) > 1:
            continue
        mid = (pt + image) / 2
        normal = (mid - pt)

        plane = g3.Plane(g3.Point(mid), g3.Vector(normal))

        planes.append(plane)

        # p.add_mesh(pv.Plane(mid, normal), color=cs[1], opacity=0.5)
        p.add_mesh(pv.Sphere(radius=0.05, center=image), color=cs[1])

# planes.append(g3.Plane.xy_plane())
# planes.append(g3.Plane.xz_plane())
# planes.append(g3.Plane.yz_plane())

halfspaces = []
for plane in planes:
    # ax + by + cz = d
    a, b, c, d = plane.general_form()

    # scipy accepts Ax + b <= 0
    # ax + by + cz - d = 0 is our plane
    # if the point doesn't lie on this side, we negate:
    # ax + by + cz - d >= 0 => -ax - by - cz + d <= 0
    A = np.array([a, b, c])
    if A @ pt <= d:
        halfspaces.append(np.array([a, b, c, -d]))
    else:
        halfspaces.append(np.array([-a, -b, -c, d]))

halfspaces = np.array(halfspaces)
halfspace_int = HalfspaceIntersection(halfspaces, pt.reshape((3,)))
hull = ConvexHull(halfspace_int.intersections)

hull_points = [g3.Point(halfspace_int.intersections[coords]) for coords in hull.vertices]
faces = []

face_segs = []
for facet in hull.simplices:
    faces.append(g3.ConvexPolygon([hull_points[i] for i in facet]))

    for i in facet:
        for j in facet:
            if i != j:
                face_segs.append(g3.Segment(hull_points[i], hull_points[j]))

cage = g3.ConvexPolyhedron(faces)

for seg in face_segs:
    p1 = seg.start_point
    p2 = seg.end_point

    # for op in sg_ops:
    #     p.add_mesh(pv.Line((p1.x, p1.y, p1.z), (p2.x, p2.y, p2.z)), color=cs[2], opacity=0.8)


# p.show()

In [158]:
from sympy import *

x1, y1, z1= symbols('x_1 y_1 z_1', real=True, positive=True)
aa, ab, ac, bb, bc, cc = symbols('aa ab ac bb bc cc', positive=True, real=True)

# metric_tensor = Matrix([
#     [aa, ab, ac],
#     [ab, bb, bc],
#     [ac, bc, cc]
# ])
metric_tensor = eye(3)

@np.vectorize
def frac(p):
    return Rational(int(p * 144), 144)

p1 = Matrix([x1, y1, z1, 1])

mats = []
dists = []
for op in sg_ops:
    for shift in shifts[:1]:
        shift_m = np.eye(4)
        shift_m[:3, 3] = shift
        mats.append(Matrix(frac(shift_m @ op.affine_matrix)))
        image = (mats[-1][:3, :] @ p1)
        dists.append((image.T @ metric_tensor @ image)[0, 0])

boundary = simplify(dists[3] - dists[12])
boundary

x_1/2 - 2*y_1 - 3*z_1/2 + 1/4

In [159]:
p, q, r, s, t, u = symbols('p q r s t u', real=True)
v1, v2, v3 = symbols('v_1 v_2 v_3', real=True)
M = Matrix([
    [p, q, r, v1],
    [q, s, t, v2],
    [r, t, u, v3],
    [0, 0, 0, 1]
])

xyz = Matrix([x1, y1, z1])

expr = expand((p1.T @ M @ p1)[0])
expr

p*x_1**2 + 2*q*x_1*y_1 + 2*r*x_1*z_1 + s*y_1**2 + 2*t*y_1*z_1 + u*z_1**2 + v_1*x_1 + v_2*y_1 + v_3*z_1 + 1

In [160]:
A, B = MatrixSymbol('A', 4, 4), MatrixSymbol('B', 4, 4)

# xyz = Matrix([x1, y1, z1])

dista = (A @ p1).T @ eye(4) @ A @ p1
distb = (B @ p1).T @ eye(4) @ B @ p1

expand(dista - distb)

Matrix([[-x_1, -y_1, -z_1, -1]])*B.T*B*Matrix([
[x_1],
[y_1],
[z_1],
[  1]]) + Matrix([[x_1, y_1, z_1, 1]])*A.T*A*Matrix([
[x_1],
[y_1],
[z_1],
[  1]])

In [161]:
A, B = sg_ops[9].affine_matrix, sg_ops[13].affine_matrix

def diff_eq(A, B):
    return A.T @ A - B.T @ B

(p1.T @ frac(diff_eq(A, B)))

Matrix([[1/4, -1/4, 1, x_1/4 - y_1/4 + z_1 + 1/8]])

In [162]:
frac(B.T @ B)

array([[1, 0, 0, -1/2],
       [0, 1, 0, -1/2],
       [0, 0, 1, -1/2],
       [-1/2, -1/2, -1/2, 7/4]], dtype=object)

In [163]:
frac(A.T @ A)

array([[1, 0, 0, -1/4],
       [0, 1, 0, -3/4],
       [0, 0, 1, 1/2],
       [-1/4, -3/4, 1/2, 15/8]], dtype=object)

In [164]:
[i for i, op in enumerate(sg_ops) if not np.allclose(op.rotation_matrix.T @ op.rotation_matrix - np.eye(3), 0)]

[]

In [165]:
from itertools import product
representatives = []


all_shifts = np.array(list(product((-1, 0, 1), repeat=3)))

def plane_eq(A, B):
    mat = diff_eq(A, B)
    if not np.allclose(mat[:3, :3], 0):
        raise ValueError('Here ya go')
    else:
        return mat[3] * np.array([2, 2, 2, 1])

for op_i in range(len(sg_ops)):
    op_m = sg_ops[op_i].affine_matrix

    should_draw = True
    for op_j, op in enumerate(sg_ops):
        for shift in all_shifts:
            shift_m = np.eye(4)
            shift_m[:3, 3] = shift
            if op_j != op_i or np.sum(np.abs(shift)) != 0:
                a, b, c, d = plane_eq(op_m, shift_m @ sg_ops[op_j].affine_matrix)
                if np.isclose([a, b, c], 0).sum() == 1:
                    continue
                elif np.allclose([a, b, c], 0) and d < 0:
                    print('Op {op_i} is dominated by op {op_j}')
                    should_draw = False
                elif np.allclose([a, b, c, d], 0) and op_j < op_i:
                    # print(f'Op {op_j} and {op_i} equivalent')
                    should_draw = False
                    pass

    if should_draw:
        representatives.append(op_i)

len(representatives)

53

In [166]:
[op for op in sg_ops if np.allclose(op.rotation_matrix, np.eye(3))]

[SymmOp(affine_matrix=array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])),
 SymmOp(affine_matrix=array([[1. , 0. , 0. , 0. ],
        [0. , 1. , 0. , 0.5],
        [0. , 0. , 1. , 0.5],
        [0. , 0. , 0. , 1. ]])),
 SymmOp(affine_matrix=array([[1. , 0. , 0. , 0.5],
        [0. , 1. , 0. , 0. ],
        [0. , 0. , 1. , 0.5],
        [0. , 0. , 0. , 1. ]])),
 SymmOp(affine_matrix=array([[1. , 0. , 0. , 0.5],
        [0. , 1. , 0. , 0.5],
        [0. , 0. , 1. , 0. ],
        [0. , 0. , 0. , 1. ]]))]

In [170]:
from itertools import product
from scipy.optimize import linprog

centerings = [op.translation_vector for op in sg_ops if np.allclose(op.rotation_matrix, np.eye(3))]

p = pv.Plotter()

for op_i, color in zip(representatives[::-1], sns.color_palette('rho_candela', len(representatives))):
    op_m = sg_ops[op_i].affine_matrix

    halfspaces = []
    should_draw = True
    for op_j, op in enumerate(sg_ops):
        for shift in all_shifts:
            shift_m = np.eye(4)
            shift_m[:3, 3] = shift
            if op_j != op_i or np.sum(np.abs(shift)) != 0:
                a, b, c, d = plane_eq(op_m, shift_m @ sg_ops[op_j].affine_matrix)
                if np.allclose([a, b, c], 0) and d < 0:
                    print('Op {op_i} is dominated by op {op_j}')
                    should_draw = False
                elif np.allclose([a, b, c, d], 0) and op_j < op_i:
                    # print(f'Op {op_j} and {op_i} equivalent')
                    # should_draw = False
                    halfspaces.append([a, b, c, d])
                elif np.allclose([a, b, c, d], 0):
                    continue
                else:
                    halfspaces.append([a, b, c, d])

    if not should_draw:
        continue

    halfspaces = np.array(halfspaces)


    norm_vector = np.reshape(np.linalg.norm(halfspaces[:, :-1], axis=1), (halfspaces.shape[0], 1))
    c = np.zeros((halfspaces.shape[1],))
    c[-1] = -1
    A = np.hstack((halfspaces[:, :-1], norm_vector))
    b = - halfspaces[:, -1:]

    res = linprog(c, A_ub=A, b_ub=b, bounds=(None, None))
    res_center = res.x[:-1]
    res_rad = res.x[-1]

    halfspace_int = HalfspaceIntersection(halfspaces, res_center)

    int_points = halfspace_int.intersections
    hull = ConvexHull(int_points)

    if op_i == representatives[0]:
        color = plt.rcParams['text.color']


    for shift, color in zip(all_shifts, sns.color_palette('rho_spectra', len(all_shifts))):
        for centering in centerings:
            if np.sum(np.square(halfspace_int.interior_point + shift + centering)) < 1:
                mesh = pv.PolyData.from_regular_faces(
                    hull.points + shift + centering,
                    hull.simplices
                )
                p.add_mesh(mesh, color=color, opacity=0.95)

p.show()

Widget(value="<iframe src='http://localhost:45491/index.html?ui=P_0x7db58e51ef50_33&reconnect=auto' style='wid…

In [168]:
import cctbx.sgtbx.direct_space_asu.reference_table as tab

max_denoms = [set(), set(), set()]

for sgnum in range(1, 231):
    asu = getattr(tab, f'asu_{sgnum:03}')()
    for pt in asu.shape_vertices():
        for i, val in enumerate(pt):
            max_denoms[i].add(val)

max_denoms

[{-3/8, -1/4, -1/8, 0, 1/8, 1/4, 1/3, 3/8, 1/2, 2/3, 3/4, 1},
 {-1/4, -1/8, 0, 1/8, 1/4, 1/3, 3/8, 1/2, 2/3, 3/4, 1},
 {-1/2, -3/8, -1/4, -1/8, 0, 1/12, 1/8, 1/6, 1/4, 1/3, 3/8, 1/2, 3/4, 1}]

In [114]:
tab.asu_230().shape_vertices()

[(-1/8, -1/8, 1/8),
 (-1/8, -1/8, 1/4),
 (-1/8, 1/8, 1/8),
 (-1/8, 1/8, 1/4),
 (0, 0, 0),
 (1/8, -1/8, 1/8),
 (1/8, -1/8, 1/4),
 (1/8, 1/8, 1/8),
 (1/8, 1/8, 1/4)]